In [1]:
import json
import pandas as pd
from datasets import load_dataset, Dataset

In [4]:
with open('./data/sample.json') as f:
    data = json.load(f)

In [5]:
# numberOfItems: 3140

# data
# 	header
# 		dialogueInfo
# 			dialogueID: dialogue 식별 번호 (str)
# 			numberOfParticipants: 참여자 수 (int)
# 			numberOfUtterances: 말 수 (int)
# 			numberOfTurns: 말 차례 수 (int)
# 			type: 대화 유형 (str)
# 			topic: 대화 주제 (str)
# 		participantsInfo: 대화 참여자 정보 (list)
# 			participantsID: 참여자 식별 아이디 (str)
# 				gender: 참여자 성별 (str)
# 				age: 참여자 연령대 (str)
# 				residentialProvince: 참여자 거주지 (str)

# 	body
# 		dialogue: 대화 목록 (list)
# 			utteranceID: 발화 ID (dialogue내 각 발화당 unique)
# 			turnID: 말차례 ID
# 			participantID: 대화 참여자 ID (대화 참여자 수와 같다)
# 			date: 날짜
# 			time: 시간
# 			utteance : 말
# 		summary: 요약문

"""
변경할 데이터
dialogueID
utterance
summary
"""

'\n변경할 데이터\ndialogueID\nutterance\nsummary\n'

In [36]:
def dict_aihub(raw_data):
    data_len = raw_data['numberOfItems']
    data = raw_data['data']
    data_dict = {'data':[]}

    for i in range(data_len):
        # 대화 식별 번호 (unique ID)
        dialogueID = data[i]['header']['dialogueInfo']['dialogueID']

        # 대화만 추출 ("발화자: 대화내용" \n로 이어붙임)
        utterances = [data[i]['body']['dialogue'][k]['utterance'] for k in range(len(data[i]['body']['dialogue']))]
        participantIDs = [data[i]['body']['dialogue'][k]['participantID'] for k in range(len(data[i]['body']['dialogue']))]
        dial = [f'{participantIDs[k]}: {utterances[k]}' for k in range(len(utterances))]
        dial = '\n'.join(dial)
        # 요약 목록
        summary = data[i]['body']['summary']
        data_dict[dialogueID] = {'dialogue': dial, 'summary': summary}
        data_dict['data'].append({'ID': dialogueID, 'dialogue': dial, 'summary': summary})

    return data_dict

In [37]:
dict_data = dict_aihub(data)

In [80]:
train_dict = {'data': dict_data['data'][:3000]}
valid_dict = {'data': dict_data['data'][3000:]}

In [81]:
# Total data
with open('./data/aihub_all.json', 'w') as f:
    json.dump(dict_data, f, ensure_ascii=False, indent=4)
# Train data
with open('./data/aihub_train.json', 'w') as f:
    json.dump(train_dict, f, ensure_ascii=False, indent=4)
# valid data
with open('./data/aihub_valid.json', 'w') as f:
    json.dump(valid_dict, f, ensure_ascii=False, indent=4)

In [82]:
d = load_dataset('json', data_files='./data/aihub_all.json', field='data')

Using custom data configuration default-13478544d65a6365


Dataset json downloaded and prepared to /opt/ml/.cache/huggingface/datasets/json/default-13478544d65a6365/0.0.0/83d5b3a2f62630efc6b5315f00f20209b4ad91a00ac586597caee3a4da0bef02. Subsequent calls will reuse this data.


In [84]:
aihub_data = load_dataset('json', data_files={'train':'./data/aihub_train.json', 'validation': './data/aihub_valid.json'}, field='data')

Using custom data configuration default-81a72d79245e17a8


Dataset json downloaded and prepared to /opt/ml/.cache/huggingface/datasets/json/default-81a72d79245e17a8/0.0.0/83d5b3a2f62630efc6b5315f00f20209b4ad91a00ac586597caee3a4da0bef02. Subsequent calls will reuse this data.


In [85]:
aihub_data

DatasetDict({
    train: Dataset({
        features: ['ID', 'dialogue', 'summary'],
        num_rows: 3000
    })
    validation: Dataset({
        features: ['ID', 'dialogue', 'summary'],
        num_rows: 140
    })
})

In [51]:
train_df = pd.DataFrame(d['train'])

In [52]:
train_df

,ID,dialogue,summary
0,43ca4319-4599-56be-a07f-6ec50db0ef4f,P01: 너~~무 앞인데\nP02: ㅋㄲㅋㅋ\nP02: #@이름#가\nP02: 겉돈...,너와 나처럼 편하고 잘 노는 아이는 아니라서 겉돈다는 느낌이 있다며 헤어졌는데 너무...
1,6e0a8762-a333-58b8-b7ce-c1511cf2c426,P01: 엄마 대단하지 않냐\nP01: 아빠랑 30년동안 같이 자고\nP02: 코 ...,아빠가 코 골고 잠꼬대하는데 30년 동안 같이 자는 엄마가 대단한 것 같다.
2,3309391b-5ca5-58ea-80c4-ff70f444502d,P01: #@이모티콘#\nP01: 그러고보니까\nP01: 이제 퇴근 30분남앗네!\...,사용한 이모티콘이 너무 귀엽다고 하니 친구가 정신없다고 쓰지 말라고 했다.
3,3465b9f4-894b-5c50-9ffe-c7f09091efa2,P01: 진짜 강아지 키우고싶다..\nP01: 독립생각없어?\nP02: 웅웅 ㅠㅠ\...,반려동물을 데려오고 싶은 마음을 털어놓는다.
4,6fe62806-14d9-5693-a5a4-05551f931349,P01: 지금은 애들이 일찍자니까\nP01: 전에 수업끝나고 애들보고간다고\nP02...,고모가 전에는 수업이 끝나고 밤에 애들을 보고 가고 요즈음(요즘)은 수업을 가기 전...
...,...,...,...
3135,1b9cc7f1-7f3d-5799-bad3-d77b7d00140a,P01: #@이름#이도 잘모르던데개\nP02: 엠티 얘기하던데ㅋㅋㅋㅋㅋㅋㅋㅋㅋ\nP...,엠티(MT)를 갈지 말지 고민하고 있으며 아마 갈 수 있을 것 같다.
3136,6d7a3b6c-6c81-572f-9184-b305e98d9392,P01: 배고파..!!\nP01: 근데 우리 어디가???\nP01: 주안?구월동?\...,선약이 있다고 말하고 일찍 나와서 주안이나 구월동 둘 중에 어디로 가야 할지 이야기한다.
3137,d437ac0b-190f-5dcb-9b7d-fa32e09a2860,P01: 몇시쯤볼까\nP02: 1시까지보자\nP02: 어디서봐야하지\nP01: 그러...,갤러리아에서 1시까지 보자고 하였다.
3138,1c9ed3f1-d89d-5dd0-aaa0-98750b598c8a,P01: 뚜섬으로 와\nP02: 점심도 못 먹고 개배고픈데ㅋㅋㅋㅋ\nP01: 그거 ...,투썸플레이스(투썸)에서 만나서 쿠폰을 사용하려 한다.


## Tokenizer

In [53]:
from transformers import AutoTokenizer

In [54]:
tokenizer = AutoTokenizer.from_pretrained(
    'gogamza/kobart-summarization',
    use_fast=True
)

In [55]:
tokenizer

PreTrainedTokenizerFast(name_or_path='gogamza/kobart-summarization', vocab_size=30000, model_max_len=1000000000000000019884624838656, is_fast=True, padding_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'sep_token': '</s>', 'pad_token': '<pad>', 'cls_token': '<s>', 'mask_token': AddedToken("<mask>", rstrip=False, lstrip=True, single_word=False, normalized=False)})

In [56]:
train_data = d['train']
inputs, targets = [],[]
for i in range(len(train_data)):
    inputs.append(train_data['dialogue'][i])
    targets.append(train_data['summary'][i])

In [57]:
model_inputs = tokenizer(inputs, max_length=1026, padding="max_length")
labels = tokenizer(targets, max_length=128, padding="max_length")

In [58]:
labels.keys()
model_inputs.keys()

dict_keys(['input_ids', 'attention_mask'])

In [61]:
tokenizer.decode(model_inputs['input_ids'][0])

'P01: 너~~무 앞인데\nP02: ᄏᄁᄏᄏ\nP02: #@이름#가\nP02: 겉돈단 생각 없지않아 조금하고있어서 조금 예민할수도\nP02: 너랑나처럼 #@기타#편하고 막 잘놀고그런건아니자너\nP02: 그와중에 놀아달라할때 씹으믄 약간의 소외감느낄껄ᄏᄏ\nP02: 근데오긴올듯\nP02: 근데 헤어졌는데\nP02: 너무 무시해섴ᄏ맘상했것다..\nP01: 아아아아\nP01: ᄋᄋ갑자기\nP01: 이해 팍 된다\nP01: 확 와닿앗음\nP01: ㅠㅠㅠㅠᅲ\nP01: 겉돈단 느낌 지금도 들려나\nP01: 이제 대놓고 괴롭히는 좋아ᄏᄏ이없는뎅\nP01: 운다 울어운다 울어근데 소외감 느꼇을만하네\nP01: #@기타# 멘탈 나갓을때니까\nP01: 아무도안들어주는거같거\nP02: 그러게\nP02: 하씨 친구랑놀다 얼핏본거여서\nP01: 나도 ncs하다가 내얘기하느<unk>웃기다ᄏ#@기타#웃기다ᄏᄏᄏ\nP02: 별거아니겠지해쏜디\nP01: 십어버림<pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><p